In [2]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l1/kfold' % DataBaseDir
OutputDir = '%s/l2/kfold' % DataBaseDir
kfold = 5
InputCols = ['lgb_l2', 'lgb_huber', 'lgb_fair', 'etr', 'en', 'knn', 'rf', 'xgb_rmse', 'lassolars', 
             'gbr_ls', 'gbr_huber', 'gbr_lad', 'rgf', 'rgf_sib', 'cb_ef']
#InputCols = ['lgb_l2', 'etr', 'knn']
strategy = 'lassolars'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    for i in range(len(InputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid = FoldValid.merge(valid[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldHoldout = FoldHoldout.merge(holdout[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldTest = FoldTest.merge(test[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)  
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ HoldoutData')
print(test_dfs[0].head(5))
print('-------------------------------------')
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    print('training fold %s begins...' % fold)
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']]
    # train
    model = linear_model.LassoLars(alpha= 0.0000001, max_iter= 100, verbose= True)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l2/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
print('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))
os.system('zip %s/%s.zip %s/%s.csv' % (SubmitDir, OutputFileName, SubmitDir, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors    lgb_l2  lgb_huber  lgb_fair  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.434408   3.496414  3.524189   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.454324   3.485764  3.448516   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  3.285386   3.251747  3.351515   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  3.329622   3.305899  3.410300   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.290475   3.168172  3.248719   

        etr        en       knn        rf  xgb_rmse  lassolars    gbr_ls  \
0  3.526076  3.466027  2.874370  3.496711  3.462120   3.458468  3.405589   
1  3.354191  3.422088  2.986891  3.413128  3.461413   3.408640  3.380286   
2  3.286988  3.258765  2.789780  3.283462  3.234076   3.260400  3.296323   
3  3.349817  3.395766  2.903560  3.307071  3.378686   3.373285  3.263758   
4  3.196220  3.178392  2.351860  3.273486  3.173649   3.183034  3.2

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.491709, holdout score 0.482719, valid length 45371
saving for 0th fold data done.
training fold 1 begins...
fold 1: valid score 0.482603, holdout score 0.483367, valid length 45371
saving for 1th fold data done.
training fold 2 begins...
fold 2: valid score 0.484693, holdout score 0.481517, valid length 45371
saving for 2th fold data done.
training fold 3 begins...
fold 3: valid score 0.489045, holdout score 0.482302, valid length 45370
saving for 3th fold data done.
training fold 4 begins...
fold 4: valid score 0.483756, holdout score 0.482169, valid length 45370
saving for 4th fold data done.
zip ../../data/l2/submit/lassolars_submit_2018-01-05.zip ../../data/l2/submit/lassolars_submit_2018-01-05.csv

CV score 0.4864, Holdout score 0.4824, Elapsed time: 22.00s

